In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import numpy as np
from collections import defaultdict
import string



In [171]:
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\trand\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\trand\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

HMM + Viterbi for Optimization

In [189]:
#get each sentence and tokenize as word 
tokenized_sentences = []
for sentence in sentences:
    words = word_tokenize(sentence)
    words = [word for word in words if word not in string.punctuation]
    tokenized_sentences.append(words)

In [191]:
def filter_punctuation(tagged_sentences):
    filtered_sentences = []
    for sentence in tagged_sentences:
        filtered_sentence = [(word, tag) for word, tag in sentence if word not in string.punctuation]
        filtered_sentences.append(filtered_sentence)
    return filtered_sentences

In [192]:
# Load the treebank corpus
tagged_sentences = treebank.tagged_sents(tagset='universal')
tagged_sentences = filter_punctuation(tagged_sentences)

# Split into training and testing data
train_data = tagged_sentences[:3000]
test_data = tagged_sentences[3000:]

# Create dictionaries to hold emission and transition probabilities
emission_counts = defaultdict(lambda: defaultdict(int))
transition_counts = defaultdict(lambda: defaultdict(int))
tag_counts = defaultdict(int)

In [193]:
# Count emissions and transitions
for sentence in train_data:
    previous_tag = '<s>'
    for word, tag in sentence:
        emission_counts[tag][word.lower()] += 1
        transition_counts[previous_tag][tag] += 1
        tag_counts[tag] += 1
        previous_tag = tag
    transition_counts[previous_tag]['</s>'] += 1


In [194]:
# Calculate emission probabilities: for the observable states
emission_prob = defaultdict(lambda: defaultdict(float))
for tag, words in emission_counts.items():
    total_count = float(tag_counts[tag])
    for word, count in words.items():
        emission_prob[tag][word] = count / total_count


In [195]:
# Calculate transition probabilities: a dictionary that tells the probability of getting a particular tag as 
#the next POS tag given that which tag had occured previously.
transition_prob = defaultdict(lambda: defaultdict(float))
for prev_tag, next_tags in transition_counts.items():
    total_count = float(sum(next_tags.values()))
    for next_tag, count in next_tags.items():
        transition_prob[prev_tag][next_tag] = count / total_count

In [196]:
for key, value in emission_prob.items():
    print(key, value)

NOUN defaultdict(<class 'float'>, {'pierre': 4.6034157344749805e-05, 'vinken': 9.206831468949961e-05, 'years': 0.00469548404916448, 'board': 0.0024398103392717398, 'director': 0.0010127514615844958, 'nov.': 0.0007825806748607467, 'mr.': 0.014546793720940938, 'chairman': 0.0018413662937899922, 'elsevier': 4.6034157344749805e-05, 'n.v.': 0.00013810247203424942, 'dutch': 4.6034157344749805e-05, 'group': 0.0022556737098927402, 'rudolph': 0.00013810247203424942, 'agnew': 4.6034157344749805e-05, 'consolidated': 9.206831468949961e-05, 'gold': 0.00023017078672374903, 'fields': 0.0005984440454817474, 'plc': 0.0003222391014132486, 'conglomerate': 4.6034157344749805e-05, 'form': 0.0006905123601712471, 'asbestos': 0.0005063757307922479, 'kent': 0.0003222391014132486, 'cigarette': 0.00018413662937899922, 'filters': 0.0003222391014132486, 'percentage': 0.0004143074161027482, 'cancer': 0.00036827325875799844, 'deaths': 0.00027620494406849883, 'workers': 0.0007825806748607467, 'researchers': 0.0005984

In [197]:
# def viterbi(sentence, transition_prob, emission_prob, tag_counts):
#     states = list(tag_counts.keys())
#     V = [{}]
#     path = {}

#     # Initialize base cases (t == 0)
#     for state in states:
#         V[0][state] = transition_prob['<s>'][state] * emission_prob[state].get(sentence[0], 1e-6)
#         path[state] = [state]

#     # Run Viterbi for t > 0
#     for t in range(1, len(sentence)):
#         V.append({})
#         newpath = {}

#         for state in states:
#             (prob, state_max) = max((V[t-1][y0] * transition_prob[y0].get(state, 1e-6) * emission_prob[state].get(sentence[t], 1e-6), y0) for y0 in states)
#             V[t][state] = prob
#             newpath[state] = path[state_max] + [state]

#         path = newpath

#     # Choose the best final state
#     (prob, state_max) = max((V[len(sentence) - 1][state], state) for state in states)
#     return path[state_max]


In [198]:
# Viterbi algorithm
def viterbi(observation_seqs, transition_prob, emission_prob, tag_counts):
    states = list(tag_counts.keys())
    num_states = len(states)
    num_obs = len(observation_seqs)
    
    # Initialize the probability matrix and the backpointer matrix
    prob_matrix = np.zeros((num_states, num_obs))
    backtrack = np.zeros((num_states, num_obs), dtype=int)
    
    # Initial probabilities
    initial_states = np.array([transition_prob['<s>'][state] for state in states])
    
    # Populate the initial column of the probability matrix
    for state_index, state in enumerate(states):
        prob_matrix[state_index, 0] = initial_states[state_index] * emission_prob[state].get(observation_seqs[0], 1e-6)
    
    # Populate the probability matrix for t > 0
    for t in range(1, num_obs):
        for state_index, state in enumerate(states):
            max_prob, max_state = max(
                (prob_matrix[prev_state_index, t-1] * transition_prob[prev_state][state] * emission_prob[state].get(observation_seqs[t], 1e-6), prev_state_index)
                for prev_state_index, prev_state in enumerate(states)
            )
            prob_matrix[state_index, t] = max_prob
            backtrack[state_index, t] = max_state
    
    # Find the most probable state sequence
    optimal_path = np.zeros(num_obs, dtype=int)
    optimal_path[-1] = np.argmax(prob_matrix[:, -1])
    
    for t in range(num_obs - 2, -1, -1):
        optimal_path[t] = backtrack[optimal_path[t + 1], t + 1]
    
    # Convert indices back to state names
    optimal_tags = [states[state_index] for state_index in optimal_path]
    
    return optimal_tags

In [199]:
# Evaluate the model on the test data
def evaluate_hmm(test_data, transition_prob, emission_prob, tag_counts):
    correct = total = 0
    for sentence in test_data:
        words, gold_tags = zip(*sentence)
        words = [word.lower() for word in words]
        predicted_tags = viterbi(words, transition_prob, emission_prob, tag_counts)
        correct += sum(p == g for p, g in zip(predicted_tags, gold_tags))
        total += len(gold_tags)
        # Print the comparison for each sentence
        print(f"Sentence: {' '.join(words)}")
        print(f"Predicted tags: {predicted_tags}")
        print(f"Actual tags:    {gold_tags}")
        print()
    return correct / total

In [200]:
test_sentence = word_tokenize("The ceremony was held at Westminster Abbey.".lower())
predicted_tags = viterbi(test_sentence, transition_prob, emission_prob, tag_counts)
print(list(zip(test_sentence, predicted_tags)))

[('the', 'DET'), ('ceremony', 'NOUN'), ('was', 'VERB'), ('held', 'VERB'), ('at', 'ADP'), ('westminster', 'NOUN'), ('abbey', 'NOUN'), ('.', 'NOUN')]


In [201]:
hmm_accuracy = evaluate_hmm(test_data, transition_prob, emission_prob, tag_counts)


Sentence: at tokyo the nikkei index of 225 selected issues which *t*-1 gained 132 points tuesday added 14.99 points to 35564.43
Predicted tags: ['ADP', 'NOUN', 'DET', 'NOUN', 'NOUN', 'ADP', 'NUM', 'NOUN', 'NOUN', 'DET', 'X', 'VERB', 'DET', 'NOUN', 'NOUN', 'VERB', 'DET', 'NOUN', 'PRT', 'VERB']
Actual tags:    ('ADP', 'NOUN', 'DET', 'NOUN', 'NOUN', 'ADP', 'NUM', 'VERB', 'NOUN', 'DET', 'X', 'VERB', 'NUM', 'NOUN', 'NOUN', 'VERB', 'NUM', 'NOUN', 'PRT', 'NUM')

Sentence: in early trading in tokyo thursday the nikkei index fell 63.79 points to 35500.64
Predicted tags: ['ADP', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'DET', 'NOUN', 'NOUN', 'VERB', 'DET', 'NOUN', 'PRT', 'VERB']
Actual tags:    ('ADP', 'ADV', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'DET', 'NOUN', 'NOUN', 'VERB', 'NUM', 'NOUN', 'PRT', 'NUM')

Sentence: wednesday 's volume on the first section was estimated *-1 at 900 million shares in line with tuesday 's 909 million
Predicted tags: ['NOUN', 'PRT', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB'

In [202]:
print(f'HMM Accuracy with Viterbi: {hmm_accuracy:.4f}')


HMM Accuracy with Viterbi: 0.9111
